In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anemia-types-classification/diagnosed_cbc_data_v4.csv


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2025-09-26 17:39:10.071148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758908350.361140      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758908350.445213      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv ('/kaggle/input/anemia-types-classification/diagnosed_cbc_data_v4.csv')

In [4]:
df.head(3)

,WBC,LYMp,NEUTp,LYMn,NEUTn,RBC,HGB,HCT,MCV,MCH,MCHC,PLT,PDW,PCT,Diagnosis
0,10.0,43.2,50.1,4.3,5.0,2.77,7.3,24.2,87.7,26.3,30.1,189.0,12.5,0.17,Normocytic hypochromic anemia
1,10.0,42.4,52.3,4.2,5.3,2.84,7.3,25.0,88.2,25.7,20.2,180.0,12.5,0.16,Normocytic hypochromic anemia
2,7.2,30.7,60.7,2.2,4.4,3.97,9.0,30.5,77.0,22.6,29.5,148.0,14.3,0.14,Iron deficiency anemia


In [5]:
df.shape

(1281, 15)

In [6]:
df.info

<bound method DataFrame.info of         WBC    LYMp   NEUTp     LYMn    NEUTn   RBC   HGB       HCT   MCV  \
0     10.00  43.200  50.100  4.30000  5.00000  2.77   7.3   24.2000  87.7   
1     10.00  42.400  52.300  4.20000  5.30000  2.84   7.3   25.0000  88.2   
2      7.20  30.700  60.700  2.20000  4.40000  3.97   9.0   30.5000  77.0   
3      6.00  30.200  63.500  1.80000  3.80000  4.22   3.8   32.8000  77.9   
4      4.20  39.100  53.700  1.60000  2.30000  3.93   0.4  316.0000  80.6   
...     ...     ...     ...      ...      ...   ...   ...       ...   ...   
1276   4.40  25.845  77.511  1.88076  5.14094  4.86  13.5   46.1526  80.7   
1277   5.60  25.845  77.511  1.88076  5.14094  4.85  15.0   46.1526  91.7   
1278   9.20  25.845  77.511  1.88076  5.14094  4.47  13.1   46.1526  88.7   
1279   6.48  25.845  77.511  1.88076  5.14094  4.75  13.2   46.1526  86.7   
1280   8.80  25.845  77.511  1.88076  5.14094  4.95  15.2   46.1526  89.7   

       MCH  MCHC    PLT        PDW      PCT

In [7]:
df.describe()

,WBC,LYMp,NEUTp,LYMn,NEUTn,RBC,HGB,HCT,MCV,MCH,MCHC,PLT,PDW,PCT
count,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.0000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000,1281.000000
mean,7.862717,25.845000,77.511000,1.880760,5.140940,4.708267,12.184551,46.1526,85.793919,32.084840,31.739149,229.981421,14.312512,0.260280
std,3.564466,7.038728,147.746273,1.335689,2.872294,2.817200,3.812897,104.8861,27.177663,111.170756,3.300352,93.019336,3.005079,0.685351
min,0.800000,6.200000,0.700000,0.200000,0.500000,1.360000,-10.000000,2.0000,-79.300000,10.900000,11.500000,10.000000,8.400000,0.010000
25%,6.000000,25.845000,71.100000,1.880760,5.100000,4.190000,10.800000,39.2000,81.200000,25.500000,30.600000,157.000000,13.300000,0.170000
50%,7.400000,25.845000,77.511000,1.880760,5.140940,4.600000,12.300000,46.1526,86.600000,27.800000,32.000000,213.000000,14.312512,0.260280
75%,8.680000,25.845000,77.511000,1.880760,5.140940,5.100000,13.500000,46.1526,90.200000,29.600000,32.900000,293.000000,14.700000,0.260280
max,45.700000,91.400000,5317.000000,41.800000,79.000000,90.800000,87.100000,3715.0000,990.000000,3117.000000,92.800000,660.000000,97.000000,13.600000


In [8]:
df.isnull().sum()

WBC          0
LYMp         0
NEUTp        0
LYMn         0
NEUTn        0
RBC          0
HGB          0
HCT          0
MCV          0
MCH          0
MCHC         0
PLT          0
PDW          0
PCT          0
Diagnosis    0
dtype: int64

In [9]:
y = df['Diagnosis']
x = df.drop('Diagnosis' , axis= 1)

In [10]:
le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
le.inverse_transform([1])

array(['Iron deficiency anemia'], dtype=object)

In [12]:
y = to_categorical(y)

In [13]:
y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y ,random_state= 41 , train_size=0.7
)

In [15]:
model=Sequential()
model.add(Dense(64,activation="relu",input_dim=len(x.columns)))
model.add(Dense(32,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(9,activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-26 17:39:26.738411: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])

In [17]:
model.fit(x_train, y_train, epochs = 128, batch_size=32, validation_split=0.2)

Epoch 1/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.2171 - loss: 7.6277 - val_accuracy: 0.2611 - val_loss: 2.5767
Epoch 2/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3115 - loss: 1.9295 - val_accuracy: 0.4167 - val_loss: 2.1280
Epoch 3/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4286 - loss: 1.6333 - val_accuracy: 0.3944 - val_loss: 1.8268
Epoch 4/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4197 - loss: 1.5909 - val_accuracy: 0.4556 - val_loss: 1.7331
Epoch 5/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4706 - loss: 1.4749 - val_accuracy: 0.4500 - val_loss: 1.8044
Epoch 6/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4775 - loss: 1.4356 - val_accuracy: 0.3889 - val_loss: 1.7417
Epoch 7/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5430 - loss: 1.2691 - val_accuracy: 0.5056 - val_loss: 1.6493
Epoch 8/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4967 - loss: 1.4060 - val_accuracy: 0.4500 - 